# 필요 함수 import

In [54]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

# 1차 : 쿠팡 상품 리스트 추출
> categoryId
- 밀키트 : 502482
- 1국탕전골 : 502483
- 2덮밥/비빔밥 : 502484
- 3스테이크/고기 502485
- 4면/파스타/감바스 502486
- 5분식 502487
- 6어린이 만들기 겸용 502489
- 7중식요리 502490
- 8기타요리 502491

In [2]:
# 쿠팡 전체 html 추출 함수
def get_coupang_item(URL, user_agt):
    headers = {"User-Agent":user_agt
               , "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}
    res = requests.get(URL, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'lxml')
    return soup

## 필요 정보만 추출
- 카테고리 대분류, 상품id, 상품only, 상품명(전체), 정가, 할인율, 판매가, 100g당_가격, 별점, 리뷰수, 품절여부

In [55]:
#카테고리 대분류 라벨링 함수
def category_number(categoryId, category):
    return category.get(categoryId)


#1차 쿠팡 상품 리스트 전처리 함수
def first_preprocessing(cg_soup, category_num):
    
    #상품id, data-item-id, data-vendor-item-id
    bady_pdt = cg_soup.select_one("a.baby-product-link")
    productId = bady_pdt['data-product-id']
    itemsId = bady_pdt['data-item-id']
    vendorItemId = bady_pdt['data-vendor-item-id']

    # 상품명
    name = cg_soup.select_one("div.name").text.replace('\n', '').strip()
    name_only = name.split(', ')[0]

    # 판매가
    price_value = cg_soup.select_one("strong.price-value").text.replace(',','')
    
    try:
        # 정가
        base_price = cg_soup.select_one("del.base-price").text.replace("\n",'').strip().replace(',','')
        # 할인율
        discount_pcg = cg_soup.select_one("span.discount-percentage").text
    except:
        base_price, discount_pcg = price_value, 0

    #100g당 가격
    try:
        unit_price = cg_soup.select_one("span.unit-price").text.replace("\n", '').strip()
    except:
        unit_price = np.NaN
    
    try:
        # 별점
        star = cg_soup.select_one("span.star > em").text
        # 리뷰수
        rating_count = cg_soup.select_one("span.rating-total-count").text
        rating_count = re.sub(r'[^0-9]','',rating_count)
    except:
        star, rating_count = 0, 0

    # 품절여부 > 품절이면 1, 아니면 0
    try:
        out_of_stock = cg_soup.select_one("div.out-of-stock").text.replace("\n", '').strip()
        out_of_stock = 1
    except:
        out_of_stock = 0
    
    #DataFrame으로 병합
    coupang_item = pd.DataFrame({'카테고리명':category_num,'상품id':productId, 'data-item-id':itemsId, 'data-vendor-item-id':vendorItemId
                                , '상품':name_only,'상품명':name, '정가':base_price, '할인율':discount_pcg, '판매가':price_value
                                , '100g당_가격':unit_price, '별점':star, '리뷰수':rating_count, '품절여부':out_of_stock}
                                , index=[0])
    return coupang_item


## 쿠팡 상품 리스트 추출 실행부

In [48]:
a = category.keys()
a = list(a)
print(a)

['502483', '502484', '502485', '502486', '502487', '502489', '502490', '502491']


In [ ]:
#product-list-paging > div > a:nth-child(6)

In [65]:
#categoryId 기반 카테고리 대분류 정의
category = {'502483':1, '502484':2, '502485':3, '502486':4, '502487':5, '502489':6, '502490':7, '502491':8}


#쿠팡 상품 리스트 페이지 html 추출
cg_url = 'https://www.coupang.com/np/categories/'
categoryId = '502485'
page = '1'

URL = cg_url+categoryId+"?page="+page
user_agt = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.46"

cg_soup_base = get_coupang_item(URL, user_agt)
cg_soup2 = cg_soup_base.select("#productList li")


#상품 리스트를 담을 빈 dataframe 생성
item_list = ['카테고리명','상품id','data-item-id','data-vendor-item-id','상품','상품명','정가',
             '할인율','판매가','100g당_가격','별점','리뷰수','품절여부']
coupang_items = pd.DataFrame(columns=item_list)


#코드 실행
for cg_soup in cg_soup2:
    coupang_item = first_preprocessing(cg_soup, category_number(categoryId, category))
    coupang_items = pd.concat([coupang_items,coupang_item], ignore_index=True)

coupang_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   카테고리명                36 non-null     object
 1   상품id                 36 non-null     object
 2   data-item-id         36 non-null     object
 3   data-vendor-item-id  36 non-null     object
 4   상품                   36 non-null     object
 5   상품명                  36 non-null     object
 6   정가                   36 non-null     object
 7   할인율                  36 non-null     object
 8   판매가                  36 non-null     object
 9   100g당_가격             35 non-null     object
 10  별점                   36 non-null     object
 11  리뷰수                  36 non-null     object
 12  품절여부                 36 non-null     object
dtypes: object(13)
memory usage: 3.8+ KB


In [66]:
coupang_items

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부
0,3,2212937714,3767825253,71752894115,곰곰 골든비프 찹스테이크 밀키트,"곰곰 골든비프 찹스테이크 밀키트, 571g, 1개",16990,5%,15990,"(100g당 2,800원)",5.0,13117,0
1,3,1592200730,2720528064,70710761564,마이셰프 마이셰프 레드와인스테이크,"마이셰프 마이셰프 레드와인스테이크, 539g, 1개",17900,11%,15900,"(100g당 2,950원)",4.5,3008,0
2,3,320547546,1025762372,5469085011,앙트레 쿠킹박스 찹스테이크 2인분,"앙트레 쿠킹박스 찹스테이크 2인분, 535g, 1개",16900,0,16900,"(100g당 3,159원)",4.5,12488,0
3,3,1431137302,2472150605,70465541145,마이셰프 와일드 홀그레인 스테이크,"마이셰프 와일드 홀그레인 스테이크, 558g, 1개",14900,0,14900,"(100g당 2,670원)",4.5,1722,0
4,3,6354980268,13389070677,80643913724,마이셰프 기사식당 돼지 불백,"마이셰프 기사식당 돼지 불백, 695g, 1개",12900,0,12900,"(100g당 1,856원)",4.5,1006,0
5,3,4550022981,5521392375,72820915162,명장의 맛 시즈닝 척아이롤 스테이크,"명장의 맛 시즈닝 척아이롤 스테이크, 350g, 1팩",14900,0,14900,"(100g당 4,257원)",4.5,5594,0
6,3,6439578446,13933872806,81183220017,딜리조이 찹스테이크 밀키트,"딜리조이 찹스테이크 밀키트, 1개",16590,9%,14990,NaN,4.5,76,0
7,3,6223984745,12470789188,79739631592,마이셰프 포크 찹스테이크,"마이셰프 포크 찹스테이크, 648g, 1개",13900,0,13900,"(100g당 2,145원)",4.5,1216,0
8,3,6091480425,11358751325,78635100077,이플원 호주산 다이닝 안심 스테이크,"이플원 호주산 다이닝 안심 스테이크, 300g, 1개",29000,0,29000,"(100g당 9,667원)",5.0,191,0
9,3,4841176772,6256834458,73552540488,곰곰 블루베리 찹스테이크 (냉장),"곰곰 블루베리 찹스테이크 (냉장), 563g, 1개",14990,10%,13490,"(100g당 2,396원)",4.5,798,0


# 2차 : 구성 정보 추출
- 메뉴이름, 구성정보

In [ ]:
# product_url = 'https://www.coupang.com/vp/products/'
# productId = '1866720935' #data-product-id
# itemsId = '3172927841'  #data-item-id
# vendorItemId = '71160443802' #data-vendor-item-id
# user_agt = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.46"
# URL = product_url+productId+'/items/'+itemsId+'/vendoritems/'+vendorItemId
# pdt_soup = get_coupang_item(URL, user_agt)
# coupang_items[['상품id','data-item-id','data-vendor-item-id']]
# for productId, itemId, vendorItemId in coupang_items[['상품id','data-item-id','data-vendor-item-id']]:
#     URL = product_url+productId+'/items/'+itemsId+'/vendoritems/'+vendorItemId
#     pdt_soup = get_coupang_item(URL, user_agt)
    
#     all = pdt_soup.find('body').children
#     cnt = 0
#     for item in all:
#         info_of_organization = 'NaN'
#         s = item.text.strip()
#         if cnt == 2:
#             info_of_organization = s
#             cnt = 0
        
#         if cnt == 1:
#             cnt = 2
        
#         if '구성 정보' in s:
#             #print(s)
#             cnt = 1
#     data_item = pd.DataFrame({'상품':productId, '구성정보':info_of_organization})

In [67]:
data_items = pd.DataFrame(columns=['상품id', '구성정보'])

for idx in range(len(coupang_items)):
    product_url = 'https://www.coupang.com/vp/products/'
    productId = coupang_items.loc[idx, '상품id'] #data-product-id
    itemsId = coupang_items.loc[idx, 'data-item-id']  #data-item-id
    vendorItemId = coupang_items.loc[idx, 'data-vendor-item-id'] #data-vendor-item-id
    URL = product_url+str(productId)+'/items/'+str(itemsId)+'/vendoritems/'+str(vendorItemId)
    
    pdt_soup = get_coupang_item(URL, user_agt)
    
    all = pdt_soup.find('body').children
    cnt = 0
    for item in all:
        s = item.text.strip()
        if cnt == 2:
            info_of_organization = s
            cnt = 3
        
        elif cnt == 1:
            cnt = 2
        
        elif '구성 정보' in s: #해당 상세페이지에 
            #print(s)
            cnt = 1
    if not cnt==3:
        info_of_organization = np.NaN
    data_item = pd.DataFrame({'상품id':productId, '구성정보':info_of_organization}, index=[0])
    data_items = pd.concat([data_items, data_item], ignore_index=True)
data_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상품id    36 non-null     object
 1   구성정보    24 non-null     object
dtypes: object(2)
memory usage: 704.0+ bytes


In [68]:
new_coupang_df = pd.merge(coupang_items, data_items)
new_coupang_df

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보
0,3,2212937714,3767825253,71752894115,곰곰 골든비프 찹스테이크 밀키트,"곰곰 골든비프 찹스테이크 밀키트, 571g, 1개",16990,5%,15990,"(100g당 2,800원)",5.0,13117,0,NaN
1,3,1592200730,2720528064,70710761564,마이셰프 마이셰프 레드와인스테이크,"마이셰프 마이셰프 레드와인스테이크, 539g, 1개",17900,11%,15900,"(100g당 2,950원)",4.5,3008,0,"소고기, 갈릭 후레이크 튀김, 새송이버섯, 꼬마양배추, 방울토마토, 버터, 시즈닝,..."
2,3,320547546,1025762372,5469085011,앙트레 쿠킹박스 찹스테이크 2인분,"앙트레 쿠킹박스 찹스테이크 2인분, 535g, 1개",16900,0,16900,"(100g당 3,159원)",4.5,12488,0,"쇠고기, 마늘 편, 적양파, 피망, 파프리카, 양송이, 파슬리, 올리브오일, 스테이..."
3,3,1431137302,2472150605,70465541145,마이셰프 와일드 홀그레인 스테이크,"마이셰프 와일드 홀그레인 스테이크, 558g, 1개",14900,0,14900,"(100g당 2,670원)",4.5,1722,0,"소고기, 엄지 새송이버섯, 쥬키니호박, 방울토마토, 올리브유, 소스, 시즈닝, 버터..."
4,3,6354980268,13389070677,80643913724,마이셰프 기사식당 돼지 불백,"마이셰프 기사식당 돼지 불백, 695g, 1개",12900,0,12900,"(100g당 1,856원)",4.5,1006,0,"돼지고기, 야채(양파, 당근, 대파), 오일, 볶음 소스로 구성되어 있습니다."
5,3,4550022981,5521392375,72820915162,명장의 맛 시즈닝 척아이롤 스테이크,"명장의 맛 시즈닝 척아이롤 스테이크, 350g, 1팩",14900,0,14900,"(100g당 4,257원)",4.5,5594,0,"척아이롤 스테이크, 서류가공품(감자), 버터, 소스로 구성되어 있습니다."
6,3,6439578446,13933872806,81183220017,딜리조이 찹스테이크 밀키트,"딜리조이 찹스테이크 밀키트, 1개",16590,9%,14990,NaN,4.5,76,0,NaN
7,3,6223984745,12470789188,79739631592,마이셰프 포크 찹스테이크,"마이셰프 포크 찹스테이크, 648g, 1개",13900,0,13900,"(100g당 2,145원)",4.5,1216,0,"채소(마늘, 적양파, 브로콜리, 컬리플라워), 돼지고기, 엔젤헤어모짜, 소스, 시즈..."
8,3,6091480425,11358751325,78635100077,이플원 호주산 다이닝 안심 스테이크,"이플원 호주산 다이닝 안심 스테이크, 300g, 1개",29000,0,29000,"(100g당 9,667원)",5.0,191,0,"안심 스테이크, 레드와인 스테이크 소스, 이즈니 버터 컵, 와사비, 스테이크 시즈닝..."
9,3,4841176772,6256834458,73552540488,곰곰 블루베리 찹스테이크 (냉장),"곰곰 블루베리 찹스테이크 (냉장), 563g, 1개",14990,10%,13490,"(100g당 2,396원)",4.5,798,0,NaN


In [69]:
new_coupang_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 0 to 35
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   카테고리명                36 non-null     object
 1   상품id                 36 non-null     object
 2   data-item-id         36 non-null     object
 3   data-vendor-item-id  36 non-null     object
 4   상품                   36 non-null     object
 5   상품명                  36 non-null     object
 6   정가                   36 non-null     object
 7   할인율                  36 non-null     object
 8   판매가                  36 non-null     object
 9   100g당_가격             35 non-null     object
 10  별점                   36 non-null     object
 11  리뷰수                  36 non-null     object
 12  품절여부                 36 non-null     object
 13  구성정보                 24 non-null     object
dtypes: object(14)
memory usage: 4.2+ KB
